<a href="https://colab.research.google.com/github/PabloMendieta03/UFV-VisualizacionDeDatos/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Histórico de Datos

1. número de Actuals y ForeCast
2. Horizonte de previsión (es el mismo para todas las previsiones).
3. Número  Países , productos...
4. Histórico de datos de actuals/forecast
5. Forecast distintios
6. Rango de Cantidades  


## Carga de la Base de Datos

In [ ]:
# Instalación de librerías
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.1 MB/s eta 0:00:00


In [ ]:
# Importación de librerías
import pandas as pd
from google.colab import drive
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [ ]:
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)

# mostrar los datos
display(data)

Mounted at /content/drive


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05
...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05


## Análisis de los Datos

¿ Número de acutals y AI_forecast ?

In [ ]:
# Contar los registros por cada tipo de escenario
conteo_escenarios = data['SCENARIO'].value_counts()

# Filtrar solo los valores para 'actuals' y 'AI_forecast'
conteo_actuals_ai_forecast = conteo_escenarios[['actual', 'AI_forecast']]

# Mostrar el resultado
print(conteo_actuals_ai_forecast)


SCENARIO
actual           900
AI_forecast    17766
Name: count, dtype: int64


¿ Número de Países ?

In [ ]:
# Obtener el número de países únicos
numero_paises = data['COUNTRY'].nunique()

# Mostrar el resultado
print(f"El número de países es: {numero_paises}")


El número de países es: 9


¿ Número de Productos ?

In [ ]:
# Obtener el número de productos únicos
numero_productos = data['SUBBRAND'].nunique()

# Mostrar el resultado
print(f"El número de productos diferentes es: {numero_productos}")


El número de productos diferentes es: 6


¿ Número de Forecast distintos?

In [ ]:
# Obtener el número de forecasts distintos
numero_forecasts_distintos = data['FORECAST'].nunique()

# Mostrar el resultado
print(f"El número de forecasts distintos es: {numero_forecasts_distintos}")

El número de forecasts distintos es: 12


¿ Horizonte de Previsión ?

In [ ]:
combinaciones_unicas = data[['COUNTRY', 'SUBBRAND', 'FORECAST']].drop_duplicates()

# Inicializar una lista para guardar los tamaños
horizontes_length = []

# Calcular el tamaño para cada combinación
for index, fila in combinaciones_unicas.iterrows():
    # Filtrar los datos
    datos_horizonte = data[ (data['COUNTRY'] == fila['COUNTRY']) &
                          (data['SUBBRAND'] == fila['SUBBRAND']) &
                          (data['FORECAST'] == fila['FORECAST']) ]

    # Guardar el tamaño en la lista
    horizontes_length.append(datos_horizonte.shape[0])

conteo_tamaños = pd.Series(horizontes_length).value_counts().reset_index()



conteo_tamaños.columns = ['Tamaño', 'Frecuencia']


fig = px.pie(conteo_tamaños,
             names='Tamaño',
             values='Frecuencia',
             title='Tamaños de Horizontes de Previsión',
             color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()



Bibliografía:
* https://plotly.com/python/time-series/

In [ ]:
# Crea la aplicación Dash
app = Dash(__name__)

# Obtener las opciones únicas para COUNTRY y SUBBRAND
paises = data['COUNTRY'].unique()
marcas = data['SUBBRAND'].unique()

# Definir el layout de la aplicación
app.layout = html.Div([
    dcc.Graph(id="time-series-chart"),
    html.P("Selecciona un país:", style={'color': 'white'}),  # Cambiar color a blanco
    dcc.Dropdown(
        id="country-dropdown",
        options=[{'label': country, 'value': country} for country in paises],
        value=paises[0],  # Valor por defecto
        clearable=False,
    ),
    html.P("Selecciona un submarca:", style={'color': 'white'}),  # Cambiar color a blanco
    dcc.Dropdown(
        id="subbrand-dropdown",
        options=[{'label': subbrand, 'value': subbrand} for subbrand in marcas],
        value=marcas[0],  # Valor por defecto
        clearable=False,
    ),
])

# Callback para actualizar el gráfico
@app.callback(
    Output("time-series-chart", "figure"),
    [Input("country-dropdown", "value"),
     Input("subbrand-dropdown", "value")]
)
def update_graph(selected_country, selected_subbrand):
    # Filtrar los datos según la selección
    filtered_data = data[(data['FORECAST'] == 'AI_P02F') &
                         (data['COUNTRY'] == selected_country) &
                         (data['SUBBRAND'] == selected_subbrand)]

    # Crear la columna de fecha
    filtered_data['DATE'] = pd.to_datetime(filtered_data[['YEAR', 'MONTH']].assign(DAY=1))

    # Agrupar por la nueva columna de fecha y sumar el amount
    aggregated_data = filtered_data.groupby('DATE').agg({'AMOUNT': 'sum'}).reset_index()

    # Graficar
    fig = px.line(aggregated_data, x='DATE', y='AMOUNT',
                  title='Cantidad Por meses',
                  labels={'AMOUNT': 'Amount', 'DATE': 'Fecha'})
    return fig

# Ejecutar el servidor
if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

## 1. Distribución de las ventas Realizadas:



### a) Por Países:

In [ ]:
# Agrupamos por país y sumamos las ventas
ventas_por_pais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Ordenamos de Mayor a Menor por número de ventas
ventas_por_pais = ventas_por_pais.sort_values(by='AMOUNT', ascending=False)

# Gráfico de Barras
fig = px.bar(ventas_por_pais, x='COUNTRY', y='AMOUNT',
             title='Distribución de Ventas por País',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Ventas Totales'},
             text_auto=True)
fig.show()

### b) Por mes y año:

In [ ]:
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupamos por mes y año y sumamos las ventas
ventas_por_mes = data.groupby(data['DATE'].dt.to_period('M'))['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Líneas
fig = px.line(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año ( Totales)',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()


In [ ]:
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupamos por país, mes y año y sumamos las ventas
ventas_por_mes = data.groupby(['COUNTRY', data['DATE'].dt.to_period('M')])['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Líneas
fig = px.line(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año por País',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              color='COUNTRY',  # Color por país
              markers=True
)

fig.show()

In [ ]:
# Gráfico de Área
fig = px.area(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año por País',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              color='COUNTRY'
)

fig.show()


### c) Por marca:

In [ ]:
# Agrupamos por submarca y sumamos las ventas
ventas_por_submarca = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Gráfico de Pastel
fig = px.pie(ventas_por_submarca, values='AMOUNT', names='SUBBRAND',
             title='Distribución de Ventas por Marca',
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Ventas Totales'},
             color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textinfo='percent+label')

fig.show()



## 2. Tendencia y Estacionalidad:

### a) Todas las ventas del país con menos ventas

País con menos ventas:

In [ ]:
# Agrupamos por país y sumamos las ventas
ventas_por_pais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Encontramos el país con menos ventas
pais_menos_ventas = ventas_por_pais.loc[ventas_por_pais['AMOUNT'].idxmin()]
print(pais_menos_ventas)


COUNTRY               Spain
AMOUNT     218493569.403429
Name: 8, dtype: object


In [ ]:
# Filtrar datos para el país con menos ventas
datos_pais = data[data['COUNTRY'] == pais_menos_ventas['COUNTRY']]
datos_pais['DATE'] = pd.to_datetime(datos_pais[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar por fecha y sumar las ventas
ventas_por_fecha = datos_pais.groupby('DATE')['AMOUNT'].sum().reset_index()


<ipython-input-20-47f20dd68fd4>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import pandas as pd
import plotly.express as px

# Agrupamos por país y sumamos las ventas
ventas_por_pais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Encontrar el país con menos ventas
pais_menos_ventas = ventas_por_pais.loc[ventas_por_pais['AMOUNT'].idxmin()]

# Filtrar datos para el país con menos ventas
datos_pais = data[data['COUNTRY'] == pais_menos_ventas['COUNTRY']]

# Asegúrate de que tienes la columna 'DATE'
datos_pais['DATE'] = pd.to_datetime(datos_pais[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar por fecha y sumar las ventas
ventas_por_fecha = datos_pais.groupby('DATE')['AMOUNT'].sum().reset_index()

# Gráfico de Líneas para visualizar la tendencia
fig_tendencia = px.line(ventas_por_fecha, x='DATE', y='AMOUNT',
                         title=f'Tendencia de Ventas en {pais_menos_ventas["COUNTRY"]}',
                         labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
                         markers=True)
fig_tendencia.show()


<ipython-input-21-c419f30c3841>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Extraer año y mes para la estacionalidad
ventas_por_fecha['YEAR'] = ventas_por_fecha['DATE'].dt.year
ventas_por_fecha['MONTH'] = ventas_por_fecha['DATE'].dt.month

# Agrupar por mes y calcular el promedio de ventas por mes
ventas_estacionalidad = ventas_por_fecha.groupby('MONTH')['AMOUNT'].mean().reset_index()

# Gráfico de Barras para visualizar la estacionalidad
fig_estacionalidad = px.bar(ventas_estacionalidad, x='MONTH', y='AMOUNT',
                             title=f'Estacionalidad de Ventas en {pais_menos_ventas["COUNTRY"]}',
                             labels={'MONTH': 'Mes', 'AMOUNT': 'Ventas Promedio'},
                             text='AMOUNT')

fig_estacionalidad.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_estacionalidad.show()


### b) Marca más vendida: